In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display
from functools import reduce
import heapq

# J01

In [ ]:
df = pd.read_csv('J01.csv', sep=' ', index_col=None, header=None, names=['depth'])

Part 1 : Nombre de fois où la profondeur augmente strictement

In [ ]:
np.sum(df.depth.diff()>0)

In [ ]:
df['depth3'] = df['depth'] + df['depth'].shift() + df['depth'].shift(2)

Part 2 : Nombre de fois où la profondeur augmente strictement sur une fenêtre de 3

In [ ]:
np.sum(df.depth3.diff()>0)

# J02

In [ ]:
df = pd.read_csv('J02.csv', sep=' ', index_col=None, header=None, names=['direction', 'unit'])

In [ ]:
df_pivot = df.pivot_table(index='direction', aggfunc='sum')

Part 1 : final horizontal position times final depth

In [ ]:
df_pivot.loc['forward', 'unit'] * (df_pivot.loc['down', 'unit'] - df_pivot.loc['up', 'unit'])

Part 2 : Other type of computation

In [ ]:
df['horizontal_move'] = df.apply(lambda row: row['unit'] if row['direction'] == 'forward' else 0, axis=1)
df['aim_move'] = df.apply(lambda row: row['unit'] if row['direction'] == 'down' else (-row['unit'] if row['direction'] == 'up' else 0), axis=1)

In [ ]:
df['horizontal_position'] = df['horizontal_move'].cumsum()
df['aim'] = df['aim_move'].cumsum()

In [ ]:
df['depth_increase'] = df.apply(lambda row: row['aim'] * row['unit'] if row['direction'] == 'forward' else 0, axis=1)

In [ ]:
df['depth'] = df['depth_increase'].cumsum()

In [ ]:
df

In [ ]:
df.iloc[-1]['horizontal_position'] * df.iloc[-1]['depth']

# J03

In [ ]:
df = pd.read_csv('J03.csv', sep=' ', index_col=None, header=None, names=['code'], dtype=str)
df['code'] = df['code'].map(lambda r : [int(x) for x in r])
df = pd.DataFrame(df['code'].to_list(), columns=[2**i for i in range(11,-1,-1)])

Part 1 : Compute Gamma (most common), Epsilon (least common)

In [ ]:
Gamma_s = (df.mean(axis=0)>0.5)
Epsilon_s = (df.mean(axis=0)<0.5)

In [ ]:
Gamma = np.sum(np.dot(Gamma_s.index, Gamma_s.values))
Epsilon = np.sum(np.dot(Epsilon_s.index, Epsilon_s.values))

In [ ]:
Gamma * Epsilon

Part 2 : Oxygen x CO2

In [ ]:
df_Oxygen = df.copy()
df_CO2 = df.copy()

In [ ]:
col = 2048
while (df_Oxygen.shape[0] > 1):
    bit_max = 1 if df_Oxygen[col].mean() >= 0.5 else 0
    df_Oxygen = df_Oxygen[df_Oxygen[col] == bit_max]
    col = col/2

In [ ]:
Oxygen = np.sum(np.dot(df_Oxygen.columns, df_Oxygen.values.T))

In [ ]:
col = 2048
while (df_CO2.shape[0] > 1):
    bit_min = 0 if df_CO2[col].mean() >= 0.5 else 1
    df_CO2 = df_CO2[df_CO2[col] == bit_min]
    col = col/2

In [ ]:
CO2 = np.sum(np.dot(df_CO2.columns, df_CO2.values.T))

In [ ]:
Oxygen * CO2

# J04

In [ ]:
df = pd.read_csv('J04.csv', sep=';', index_col=None, header=None, names=['code'], dtype=str)
tirage = [int(x) for x in list(df.iloc[0].values)[0].split(',')]
df = df.drop(index=0).reset_index(drop=True)
df['code'] = df['code'].map(lambda row: [int(x) for x in row.split(' ') if len(x)>0])
df = pd.DataFrame(df['code'].to_list(), columns=['c' + str(i) for i in range(5)])

Part 1 : Find the first winning board and its score

In [ ]:
df2 = df.copy()

In [ ]:
win_board = None
win_x = None
for x in tirage:
    df2[df2 == x] = None
    if np.any(np.sum(df2.isna(), axis=1) == 5):
        win_board = np.where(np.sum(df2.isna(), axis=1) == 5)[0][0] // 5
        win_x = x
        break
    if np.any(df2.pivot_table(index=df2.index // 5, aggfunc=(lambda row : np.sum(row.isna()) == 5))):
        win_board = np.where(df2.pivot_table(index=df2.index // 5, aggfunc=(lambda row : np.sum(row.isna()) == 5)))[0][0]
        win_x = x
        break

In [ ]:
np.sum(df2.iloc[(5*win_board):(5*win_board+5)]).sum() * win_x

Part 2 : Find the last winning board

In [ ]:
df2 = df.copy()

In [ ]:
win_board_remaining = list(range(100))
win_x = None
for x in tirage:
    df2[df2 == x] = None
    lines = (np.sum(df2.isna(), axis=1) == 5)
    if np.any(lines):
        for where_x in lines[lines].index:
            win_board = where_x // 5
            win_board_remaining.remove(win_board)
            df2 = df2.drop(index=range((5*win_board),(5*win_board+5)))
    df_pivot = df2.pivot_table(index=df2.index // 5, aggfunc=(lambda row : np.sum(row.isna()) == 5))
    if np.any(df_pivot):
        for win_board in df_pivot[df_pivot.any(axis=1)].index:
            win_board_remaining.remove(win_board)
            df2 = df2.drop(index=range((5*win_board),(5*win_board+5)))
    if len(win_board_remaining) == 1:
        break


In [ ]:
for x in tirage:
    df2[df2 == x] = None
    lines = (np.sum(df2.isna(), axis=1) == 5)
    if np.any(lines):
        win_x = x
        break
    df_pivot = df2.pivot_table(index=df2.index // 5, aggfunc=(lambda row : np.sum(row.isna()) == 5))
    if np.any(df_pivot):
        win_x = x
        break

In [ ]:
df2.sum().sum()*win_x

# J05

In [ ]:
df = pd.read_csv('J05.csv', sep=' -> ', index_col=None, header=None, names=['in', 'out'], dtype=str, engine='python')
df['in'] = df['in'].map(lambda row: [int(x) for x in row.split(',') if len(x)>0])
df['out'] = df['out'].map(lambda row: [int(x) for x in row.split(',') if len(x)>0])
df = pd.concat([pd.DataFrame(df['in'].to_list(), columns=['x1','y1']), 
                pd.DataFrame(df['out'].to_list(), columns=['x2','y2'])],
               axis=1)

Part 1

In [ ]:
# df.max() --> 989
matrix = np.zeros([1000, 1000])
for _, row in df.iterrows():
    if row.x1 == row.x2:
        for y in range(min(row.y1, row.y2), max(row.y1, row.y2)+1):
            matrix[row.x1, y] = matrix[row.x1, y] + 1
    elif row.y1 == row.y2:
        for x in range(min(row.x1, row.x2), max(row.x1, row.x2)+1):
            matrix[x, row.y1] = matrix[x, row.y1] + 1

In [ ]:
np.sum(matrix > 1)

Part 2

In [ ]:
# df.max() --> 989
matrix = np.zeros([1000, 1000])
for _, row in df.iterrows():
    if row.x1 == row.x2:
        for y in range(min(row.y1, row.y2), max(row.y1, row.y2)+1):
            matrix[row.x1, y] = matrix[row.x1, y] + 1
    elif row.y1 == row.y2:
        for x in range(min(row.x1, row.x2), max(row.x1, row.x2)+1):
            matrix[x, row.y1] = matrix[x, row.y1] + 1
    else:
        pente = int((row.y2 - row.y1) / (row.x2 - row.x1))
        if row.x1 < row.x2:
            xmin = row.x1
            xmax = row.x2
            ymin = row.y1
        else:
            xmin = row.x2
            xmax = row.x1
            ymin = row.y2
        for x in range(xmin, xmax+1):
            matrix[x, ymin + pente * (x - xmin)] = matrix[x, ymin + pente * (x - xmin)] + 1

In [ ]:
np.sum(matrix > 1)

# J06

In [ ]:
df = pd.read_csv('J06.csv', sep=',', index_col=None, header=None).T
df['n'] = df[0]

In [ ]:
df_pivot = df.pivot_table(index='n', aggfunc='count')
df_pivot.loc[0,0] = 0
df_pivot.loc[6,0] = 0
df_pivot.loc[7,0] = 0
df_pivot = df_pivot.sort_index()

Part 1 : n=80 / Part 2 : n=256

In [ ]:
n = 256
for d in range(0,n):
    df_pivot[d+1] = df_pivot[d].shift(-1).fillna(0)
    df_pivot.loc[6,d+1] += df_pivot.loc[0,d]
    df_pivot.loc[8,d+1] = df_pivot.loc[0,d]
    

In [ ]:
df_pivot.sum()[n]

# J07

In [ ]:
df = pd.read_csv('J07.csv', sep=',', index_col=None, header=None).T
df = df[0].to_list()

Part 1

In [ ]:
np.sum(np.abs(df - np.median(df)))

Part 2

In [ ]:
test1 = np.sum(np.abs(df - np.floor(np.mean(df))) * (np.abs(df - np.floor(np.mean(df))) + 1))/2
test2 = np.sum(np.abs(df - np.ceil(np.mean(df))) * (np.abs(df - np.ceil(np.mean(df))) + 1))/2
min(test1, test2)

# J08

In [ ]:
df = pd.read_csv('J08.csv', sep=' ', index_col=None, header=None, 
                 names=['i'+str(i) for i in range(10)]+['s']+['o'+str(i) for i in range(4)])
df.drop(columns='s', inplace=True)

In [ ]:
for c in df.columns:
    df[c] = df[c].map(lambda x: ''.join(sorted(x)))

Part 1 : nombre de 1, 4, 7, 8 identifiables juste sur la longueur

In [ ]:
n_1478 = 0
for i in range(4):
    n_1478 += np.sum(df['o'+str(i)].str.len()==2)
    n_1478 += np.sum(df['o'+str(i)].str.len()==4)
    n_1478 += np.sum(df['o'+str(i)].str.len()==3)
    n_1478 += np.sum(df['o'+str(i)].str.len()==7)
    

In [ ]:
n_1478

Part 2 : déchiffrement

longueur 2 : 1
longueur 3 : 7
longueur 4 : 4
longueur 5 : 2, 3, 5
longueur 6 : 0, 6, 9
longueur 7 : 8

In [ ]:
for i in range(10):
    df['pos'+str(i)] = None
for i in range(4):
    df['res_o'+str(i)] = None

In [ ]:
def find_pos_by_length(row, length):
    for i in range(10):
        if len(row['i'+str(i)])==length:
            return i
    
df['pos1'] = df.apply(lambda row: find_pos_by_length(row, 2), axis=1)
df['pos4'] = df.apply(lambda row: find_pos_by_length(row, 4), axis=1)
df['pos7'] = df.apply(lambda row: find_pos_by_length(row, 3), axis=1)
df['pos8'] = df.apply(lambda row: find_pos_by_length(row, 7), axis=1)

In [ ]:
def find_pos_by_length_and_contains_car(row, length, car_in=None, car_out=None):
    if car_in is None:
        car_to_find = ''
    else:
        car_to_find = row['i'+str(row['pos'+str(car_in)])]
    if car_out is None:
        car_to_exc = 'z'
    else:
        car_to_exc = row['i'+str(row['pos'+str(car_out)])]
    for i in range(10):

        if (len(row['i'+str(i)])==length) & np.all([x in row['i'+str(i)] for x in car_to_find]) \
                                        & (np.all([x in row['i'+str(i)] for x in car_to_exc])==False):
            return i  
        
df['pos9'] = df.apply(lambda row: find_pos_by_length_and_contains_car(row, length=6, car_in=4), axis=1)
df['pos0'] = df.apply(lambda row: find_pos_by_length_and_contains_car(row, length=6, car_in=1, car_out=4), axis=1)
df['pos6'] = df.apply(lambda row: find_pos_by_length_and_contains_car(row, length=6, car_out=1), axis=1)
df['pos3'] = df.apply(lambda row: find_pos_by_length_and_contains_car(row, length=5, car_in=1), axis=1)

In [ ]:
def find_pos_by_length_and_sub_car(row, length, car_sub=None):
    if car_sub is None:
        car_to_find = 'abcdefg'
    else:
        car_to_find = row['i'+str(row['pos'+str(car_sub)])]
    for i in range(10):
        if (len(row['i'+str(i)])==length) & np.all([x in car_to_find for x in row['i'+str(i)]]):
            return i
    print('________')
    
df['pos5'] = df.apply(lambda row: find_pos_by_length_and_sub_car(row, length=5, car_sub=6), axis=1)

In [ ]:
def find_last_pos(row):
    found_pos = [row['pos'+str(x)] for x in range(10)]
    for x in range(10):
        if x not in found_pos:
            return x

df['pos2'] = df.apply(lambda row: find_last_pos(row), axis=1)

In [ ]:
def dechiffre(row, code):
    for i in range(10):
        if row['i'+str(row['pos'+str(i)])] == code:
            return i

for i in range(4):
    df['res_o'+str(i)] = df.apply(lambda row:dechiffre(row, row['o'+str(i)]), axis=1)

In [ ]:
1000 * df.res_o0.sum() + 100 * df.res_o1.sum() + 10 * df.res_o2.sum() + 1 * df.res_o3.sum()

# J09

In [ ]:
df = pd.read_csv('J09.csv', sep=' ', index_col=None, header=None, names=['x'])
df['x'] = df['x'].apply(lambda x:list(x))
df = pd.DataFrame(df['x'].to_list(), columns=['x'+str(i) for i in range(100)])
df = df.astype('int32', copy=False)
mat = df.to_numpy()

In [ ]:
mat2 = mat.copy()
row_to_be_added = np.array(mat.shape[0] * [9])
col_to_be_added = np.atleast_2d(np.array((mat.shape[1] + 2) * [9])).T
mat2 = np.vstack ((row_to_be_added, mat2, row_to_be_added) )
mat2 = np.hstack ((col_to_be_added, mat2, col_to_be_added) )

Part 1

In [ ]:
score = 0

for i in range(1,mat2.shape[0]-1):
    for j in range(1,mat2.shape[0]-1):
        if (mat2[i,j] < mat2[i+1,j]) & (mat2[i,j] < mat2[i-1,j]) & (mat2[i,j] < mat2[i,j+1]) & (mat2[i,j] < mat2[i,j-1]):
            score = score + mat2[i,j]+1
            

In [ ]:
score

Part 2

In [ ]:
mat3 = np.empty((mat2.shape[0],mat2.shape[1]))
mat3[:] = False

In [ ]:
def search_basins_size(i,j):
    mat3[i, j] = True
    if mat2[i,j] == 9:
        return 0
    n_size = 1
    for i_next in [i-1,i+1]:
        if not(mat3[i_next, j]):
            n_size = n_size + search_basins_size(i_next, j)
    for j_next in [j-1,j+1]:
        if not(mat3[i, j_next]):
            n_size = n_size + search_basins_size(i, j_next)                
    return n_size


In [ ]:
basins = {}

for i in range(1,mat2.shape[0]-1):
    for j in range(1,mat2.shape[0]-1):
        if (mat2[i,j] < mat2[i+1,j]) & (mat2[i,j] < mat2[i-1,j]) & (mat2[i,j] < mat2[i,j+1]) & (mat2[i,j] < mat2[i,j-1]):
            basins[(i,j)] = search_basins_size(i,j)
            

In [ ]:
basins = {k: v for k, v in sorted(basins.items(), key=lambda item: -item[1])}
val = list(basins.values())
val[0] * val[1] * val[2]

# J10 

In [ ]:
df = pd.read_csv('J10.csv', sep=' ', index_col=None, header=None, names=['chunk'])

In [ ]:
c_ins = ['(', '[', '{', '<']
dict_in_out = {'(':')', '[':']', '{':'}', '<':'>'}

def search_for_illegal(x):
    expected_closure = []
    for c in x:
        if c in c_ins:
            expected_closure.append(dict_in_out[c])
        else:
            if expected_closure.pop() != c:
                return c
    return None

Part 1

In [ ]:
df['illegal'] = df['chunk'].map(lambda x : search_for_illegal(x))

In [ ]:
df['score_illegal'] = df['illegal'].map({')':3,']':57,'}':1197,'>':25137})
df['score_illegal'].sum()

Part 2

In [ ]:
def search_for_completion(x):
    expected_closure = []
    for c in x:
        if c in c_ins:
            expected_closure.append(dict_in_out[c])
        else:
            if expected_closure.pop() != c:
                return []
    return expected_closure

In [ ]:
df['completion'] = df['chunk'].map(lambda x : search_for_completion(x))

In [ ]:
dict_score_completion = {')':1, ']':2, '}': 3, '>':4}

def get_score_completion(l):
    score = 0
    for c in reversed(l):
        score = 5 * score + dict_score_completion[c]
    return (None if score==0 else score)

In [ ]:
df['score_completion'] = df['completion'].map(lambda x : get_score_completion(x))
df['score_completion'].median()

# J11

In [ ]:
df = pd.read_csv('J11.csv', sep=' ', index_col=None, header=None, names=['x'], dtype=str)
n_cols = len(df.loc[0,'x'])
df['x'] = df['x'].map(lambda r : [int(x) for x in r])
df = pd.DataFrame(df['x'].to_list(), columns=range(n_cols))
mat = df.to_numpy()

Part 1

In [ ]:
mat2 = mat.copy()

In [ ]:
n_steps = 100
n_flashs = 0

for step in range(n_steps):
    mat2[:] = mat2[:] + 1
    while np.any(mat2 >= 10):
        ind = zip(*np.where(mat2>=10))
        for i,j in ind:
            mat2[i,j] = 0
            n_flashs += 1
            for i_cur in [i-1,i,i+1]:
                for j_cur in [j-1,j,j+1]:
                    if (i_cur >= 0) & (j_cur >= 0) & (i_cur < mat2.shape[0]) \
                                    & (j_cur < mat2.shape[1]) & (((i_cur == i) & (j_cur == j)) == False):
                        if mat2[i_cur, j_cur] > 0:
                            mat2[i_cur, j_cur] += 1

In [ ]:
n_flashs

Part 2

In [ ]:
mat3 = mat.copy()

In [ ]:
n_steps = 1_000_000

for step in range(n_steps):
    if np.all(mat3 == mat3.mean()):
        break
    mat3[:] = mat3[:] + 1
    while np.any(mat3 >= 10):
        ind = zip(*np.where(mat3>=10))
        for i,j in ind:
            mat3[i,j] = 0
            for i_cur in [i-1,i,i+1]:
                for j_cur in [j-1,j,j+1]:
                    if (i_cur >= 0) & (j_cur >= 0) & (i_cur < mat3.shape[0]) \
                                    & (j_cur < mat3.shape[1]) & (((i_cur == i) & (j_cur == j)) == False):
                        if mat3[i_cur, j_cur] > 0:
                            mat3[i_cur, j_cur] += 1

step

# J12

In [ ]:
df = pd.read_csv('J12.csv', sep='-', index_col=None, header=None, names=['cave1', 'cave2'])
caves = dict.fromkeys(set(df['cave1'].to_list() + df['cave2'].to_list()))
for k in caves:
    caves[k] = []

In [ ]:
for _, row in df.iterrows():
    if row.cave2 not in caves[row.cave1]: caves[row.cave1].append(row.cave2)
    if row.cave1 not in caves[row.cave2]: caves[row.cave2].append(row.cave1)


Part 1 : small caves can be visited only once

In [ ]:
found_paths = []

In [ ]:
def parcours_path_in_deph(my_path):
    last_item = my_path[-1]
    for next_item in caves[last_item]:
        if next_item == 'end':
            found_paths.append(my_path+['end'])
        elif next_item.islower() & (next_item in my_path):
            do_nothing = 0
        else:
            parcours_path_in_deph(my_path + [next_item])

In [ ]:
parcours_path_in_deph(['start'])
len(found_paths)

Part 2 : only one small cave can be visited twice

In [ ]:
import sys
sys.setrecursionlimit(50_000)

In [ ]:
found_paths = []

In [ ]:
def parcours_path_in_deph_with_joker(my_path, joker_has_been_used):
    last_item = my_path[-1]
    for next_item in caves[last_item]:
        if next_item == 'end':
            found_paths.append(my_path+['end'])
        elif next_item.islower() & (next_item in my_path):
            if (next_item in ['start', 'end']) | joker_has_been_used:
                do_nothing = 0
            else:
                parcours_path_in_deph_with_joker(my_path + [next_item], True)
        else:
            parcours_path_in_deph_with_joker(my_path + [next_item], joker_has_been_used)

In [ ]:
parcours_path_in_deph_with_joker(['start'], False)
len(found_paths)

# J13

In [ ]:
df = pd.read_csv('J13.csv', sep=',', index_col=None, header=None, names=['x','y'], dtype=str)
first_na = np.where(df.y.isna())[0][0]
folds = df.x.iloc[first_na:].values
df = df.iloc[:first_na]
df = df.astype('int32', copy=False)

Part 1 : First fold

In [ ]:
df2 = df.copy()

In [ ]:
fold = folds[0]
fold_axis = fold[11]
fold_value = int(fold[13:])

In [ ]:
df2[fold_axis] = df2[fold_axis].map(lambda x : x if x < fold_value else fold_value - (x-fold_value))

In [ ]:
df2.groupby(['x', 'y']).ngroups

Part 2

In [ ]:
df3 = df.copy()

In [ ]:
for fold in folds:
    fold_axis = fold[11]
    fold_value = int(fold[13:])
    df3[fold_axis] = df3[fold_axis].map(lambda x : x if x < fold_value else fold_value - (x-fold_value))

In [ ]:
max_x = df3.x.max()
max_y = df3.y.max()

In [ ]:
mat = np.zeros((max_y+1, max_x+1))

for _, row in df3.iterrows():
    mat[row.y, row.x] = 1

In [ ]:
plt.imshow(mat, interpolation='nearest')
plt.show()

# J14

In [ ]:
df = pd.read_csv('J14.csv', sep=' -> ', index_col=None, header=None, names=['couple','insert'], dtype=str)
polymer0 = df.loc[0,'couple']
df = df.loc[1:].reset_index(drop=True)
dict_transfo = dict(zip(df['couple'].to_list(), df['insert'].to_list()))

In [ ]:
def compute_one_step(polymer):
    n = len(polymer)
    n_added = 0
    for i in range(n-1):
        if polymer[n_added+i:n_added+i+2] in dict_transfo:
            polymer = polymer[:n_added+i+1] + dict_transfo[polymer[n_added+i:n_added+i+2]] + polymer[n_added+i+1:]
            n_added += 1
    return polymer

Part 1

In [ ]:
n_steps = 10

In [ ]:
polymer = polymer0
for i in range(n_steps):
    polymer = compute_one_step(polymer)

In [ ]:
dict_count_letters = dict.fromkeys(list(set(polymer)), 0)
for c in dict_count_letters:
    dict_count_letters[c] = polymer.count(c)
dict_count_letters = {k: v for k, v in sorted(dict_count_letters.items(), key=lambda item: item[1])}

In [ ]:
list(dict_count_letters.values())[-1] - list(dict_count_letters.values())[0]

Part 2

In [ ]:
n_steps = 40

In [ ]:
all_pairs = set()
n = len(polymer0)
for i in range(n-1):
    all_pairs.add(polymer0[i:i+2])
for k, v in dict_transfo.items():
    all_pairs.add(k)
    all_pairs.add(k[0]+v)
    all_pairs.add(v+k[1])
all_pairs = list(all_pairs)
n_pairs = len(all_pairs)
dict_place_pairs = {}
for i, p in enumerate(all_pairs):
    dict_place_pairs[p] = i

In [ ]:
mat_transfo = np.zeros((n_pairs, n_pairs))
statut = np.zeros((1, n_pairs))

In [ ]:
for i in range(n-1):
    statut[0,dict_place_pairs[polymer0[i:i+2]]] += 1

In [ ]:
for k, v in dict_transfo.items():
    mat_transfo[dict_place_pairs[k], dict_place_pairs[k[0]+v]] = 1
    mat_transfo[dict_place_pairs[k], dict_place_pairs[v+k[1]]] = 1
mat_transfo = mat_transfo.T

In [ ]:
for i in range(n_steps):
    statut = mat_transfo.dot(statut.T).T

In [ ]:
dict_count_letters = {}
for i in range(n_pairs):
    if statut[0,i] > 0:
        for j in [0,1]:
            dict_count_letters[all_pairs[i][j]] = statut[0,i] \
                        + (dict_count_letters[all_pairs[i][j]] if all_pairs[i][j] in dict_count_letters else 0)

dict_count_letters[polymer0[0]] += 1
dict_count_letters[polymer0[-1]] += 1
for k in dict_count_letters:
        dict_count_letters[k] /= 2

dict_count_letters = {k: v for k, v in sorted(dict_count_letters.items(), key=lambda item: item[1])}

In [ ]:
list(dict_count_letters.values())[-1] - list(dict_count_letters.values())[0]

# J15

In [ ]:
df = pd.read_csv('J15.csv', sep=' ', index_col=None, header=None, names=['code'], dtype=str)
df['code'] = df['code'].map(lambda r : [int(x) for x in r])
n_cols = len(df.loc[0,'code'])
df = pd.DataFrame(df['code'].to_list(), columns=range(n_cols))

Part 1

In [ ]:
def dijkstra(df, s_dep):
    distances = {(x, y) : np.Inf for y in range(df.shape[1]) for x in range(df.shape[0])}
    pile = [(0, s_dep)]
    
    while pile:
        distance, (x, y) = heapq.heappop(pile)
        if distance > distances[(x,y)]:
            continue
        
        if (x,y) == (df.shape[0]-1, df.shape[1]-1):
            return distance
        
        for neighbor in [(x+1,y), (x-1,y), (x,y+1), (x,y-1)]:
            if neighbor in distances:
                if distance + df.loc[neighbor[0], neighbor[1]] < distances[neighbor]:
                    distances[neighbor] = distance + df.loc[neighbor[0], neighbor[1]]
                    heapq.heappush(pile, (distances[neighbor], neighbor))
                    
    return distances[(df.shape[0]-1, df.shape[1]-1)]
        

In [ ]:
dijkstra(df, (0,0))

Part 2 : première implémentation de Dijkstra remplacé par une implantation par file par arbre binaire heapq

In [ ]:
df2 = df.copy()

for i in range(1,5):
    for c in df.columns:
        df2[c + i * df.shape[1]] = df2[c + (i-1) * df.shape[1]].map(lambda x : x+1 if x<9 else 1)

for i in range(1,5):
    for c in range(df.shape[0]):
        df2.loc[c + i * df.shape[0]] = df2.loc[c + (i-1) * df.shape[0]].map(lambda x : x+1 if x<9 else 1)

dijkstra(df2, (0,0))

# J16

In [ ]:
df = pd.read_csv('J16.csv', sep=' ', index_col=None, header=None, names=['code'], dtype=str)
code = df.loc[0,'code']

In [ ]:
code_bit = bin(int(code, 16))[2:].zfill(4 * len(code))


In [ ]:
def decode_litteral_paquet(paquet_to_end):
    litteral = ''
    debut_paquet = 0
    n_bloc = 0
    while True:
        litteral = litteral + paquet_to_end[debut_paquet+1:debut_paquet+5]
        n_bloc += 1 
        if (paquet_to_end[debut_paquet] == '0'):
            break
        debut_paquet = debut_paquet + 5
    litteral = int(litteral, 2)
    
    curseur = 5 * n_bloc
    return litteral, curseur
    

In [ ]:
def decode_operateur_paquet(paquet_to_end):
    type_length = paquet_to_end[0]
    if type_length == '0':
        total_length_in_bits = int(paquet_to_end[1:16], 2)
        nb_sub_paquets = None
        curseur = 16
    else:
        total_length_in_bits = None
        nb_sub_paquets = int(paquet_to_end[1:12], 2)
        curseur = 12

    return type_length, total_length_in_bits, nb_sub_paquets, curseur

Part 1

In [ ]:
n_bits = len(code_bit)
cur_bit = 0
som_paquet_version = 0
while (cur_bit < n_bits) & np.any([x != '0' for x in code_bit[cur_bit:]]):
    #cur_bit indique toujours le début d'un paquet
    paquet_version = int(code_bit[cur_bit:cur_bit+3], 2)
    paquet_id = int(code_bit[cur_bit+3:cur_bit+6], 2)
    som_paquet_version += paquet_version
    
    if paquet_id == 4: #litteral
        _, cur_bit_new = decode_litteral_paquet(code_bit[cur_bit+6:])
    else: #operateur
        type_length, total_length_in_bits, nb_sub_paquets, cur_bit_new = decode_operateur_paquet(code_bit[cur_bit+6:])
    cur_bit += cur_bit_new+6

In [ ]:
som_paquet_version

Part 2

In [ ]:
n_bits = len(code_bit)
cur_bit = 0
list_nodes = {}
n_nodes = 0

while (cur_bit < n_bits) & np.any([x != '0' for x in code_bit[cur_bit:]]):
    #cur_bit indique toujours le début d'un paquet
    paquet_version = int(code_bit[cur_bit:cur_bit+3], 2)
    paquet_id = int(code_bit[cur_bit+3:cur_bit+6], 2)
    
    if paquet_id == 4: #litteral
        value, cur_bit_new = decode_litteral_paquet(code_bit[cur_bit+6:])
        type_length = None
        total_length_in_bits = None
        nb_sub_paquets = None
    else: #operateur
        value = None
        type_length, total_length_in_bits, nb_sub_paquets, cur_bit_new = decode_operateur_paquet(code_bit[cur_bit+6:])
    cur_bit += cur_bit_new+6
    list_nodes[n_nodes] = (paquet_version, paquet_id, 
                           value, 
                           type_length, total_length_in_bits, nb_sub_paquets, 
                           cur_bit_new+6, [], [])
    n_nodes += 1

In [ ]:
for node in range(n_nodes-1, -1, -1):
    if list_nodes[node][1] == 4:
        continue
    if list_nodes[node][3] == '0': # longueur en bits
        length_added = 0
        for node_next in range(node+1, n_nodes):
            if (node_next in list_nodes[node][8]) == False:
                list_nodes[node][7].append(node_next)
                for c in list_nodes[node_next][8] + [node_next]:
                    if( c in list_nodes[node][8]) == False:
                        list_nodes[node][8].append(c)
            length_added += list_nodes[node_next][6]
            if length_added >= list_nodes[node][4]:
                break
    else: #longueur en paquets
        n_nodes_added = 0
        for node_next in range(node+1, n_nodes):
            if (node_next in list_nodes[node][8]) == False:
                list_nodes[node][7].append(node_next)
                for c in list_nodes[node_next][8] + [node_next]:
                    if( c in list_nodes[node][8]) == False:
                        list_nodes[node][8].append(c)
                n_nodes_added += 1
            if n_nodes_added >= list_nodes[node][5]:
                break

In [ ]:
#Remove doublons
for node in range(n_nodes):
    l = list_nodes[node][7].copy()
    for node_next in l:
        for c in list_nodes[node_next][7]:
            if c in list_nodes[node][7]:
                list_nodes[node][7].remove(c)

In [ ]:
dict_values = {}
for node in range(n_nodes-1, -1, -1):
    if list_nodes[node][1] == 4:
        value = list_nodes[node][2]
    elif list_nodes[node][1] == 0:
        somme = 0
        for node_next in list_nodes[node][7]:
            p = dict_values[node_next]
            somme += p
        value = somme        
    elif list_nodes[node][1] == 1:
        prod = 1
        for node_next in list_nodes[node][7]:
            prod *= dict_values[node_next]
        value = prod
    elif list_nodes[node][1] == 2:
        min_v = np.inf
        for node_next in list_nodes[node][7]:
            min_v = min(min_v, dict_values[node_next])
        value = min_v
    elif list_nodes[node][1] == 3:
        max_v = 0
        for node_next in list_nodes[node][7]:
            max_v = max(max_v, dict_values[node_next])
        value = max_v
    elif list_nodes[node][1] == 5:
        if (dict_values[list_nodes[node][7][0]] > dict_values[list_nodes[node][7][1]]):
            value = 1
        else:
            value = 0
    elif list_nodes[node][1] == 6:
        if (dict_values[list_nodes[node][7][0]] < dict_values[list_nodes[node][7][1]]):
            value = 1
        else:
            value = 0
    elif list_nodes[node][1] == 7:
        if (dict_values[list_nodes[node][7][0]] == dict_values[list_nodes[node][7][1]]):
            value = 1
        else:
            value = 0
    dict_values[node] = value

In [ ]:
dict_values[0]

# J17

target area: x=155..182, y=-117..-67

In [ ]:
target_xmin = 155
target_xmax = 182
target_ymin = -117
target_ymax = -67

In [ ]:
vx_min = int((1+ np.sqrt(8*target_xmin)) / 2) - 1
vx_max = target_xmax
vy_min = target_ymin-1
vy_max = abs(vy_min)

In [ ]:
def calcule_trajectoire(v0x, v0y):
    #On suppose v0x positif
    #On vérifie si x atteint au moins le mini
    x_max_atteint = v0x**2 + v0x * (v0x - 1) / 2
    if x_max_atteint < target_xmin:
        return np.NaN
    
    n = 0
    x = 0
    y = 0
    y_max = 0
    vx = v0x
    vy = v0y
    while True:
        if (x > target_xmax) | (y < target_ymin):
            return False, np.NaN
        y_max = max(y_max, y)
        if (target_xmin <= x <= target_xmax) & (target_ymin <= y <= target_ymax):
            return True, y_max
        n += 1
        x += vx
        y += vy
        vx = vx-1 if vx>0 else 0
        vy -= 1

Part 1

In [ ]:
y_max = 0
for vx in range(vx_min, vx_max+1):
    for vy in range(vy_min, vy_max+1):
        _, y = calcule_trajectoire(vx, vy)
        if (y>y_max):
            y_max = y

In [ ]:
y_max

Part 2

In [ ]:
n_correct = 0
for vx in range(vx_min, vx_max+1):
    for vy in range(vy_min, vy_max+1):
        s,_ = calcule_trajectoire(vx, vy)
        if s:
            n_correct += 1

In [ ]:
n_correct

# J18

Code trouvé sur Reddit - weekend de noel

In [ ]:
df = pd.read_csv('J18.csv', sep=' ', index_col=None, header=None, names=['snailfish'], dtype=str)
df

Part 1

In [ ]:
def parse_numbers(lines):
    for line in lines:
        it = iter(line)
        if next(it) != '[':
            raise ValueError
        yield parse_number_rec(it)

def parse_number_rec(it):
    c = next(it)
    if c == '[':
        left = parse_number_rec(it)
        c = next(it)
    else:
        left, c = parse_number_regular(c, it)
    if c != ',':
        raise ValueError
    c = next(it)
    if c == '[':
        right = parse_number_rec(it)
        c = next(it)
    else:
        right, c = parse_number_regular(c, it)
    if c != ']':
        raise ValueError
    return (left, right)

def parse_number_regular(c, it):
    n = 0
    while c in '0123456789':
        n *= 10
        n += int(c, 10)
        c = next(it)
    return n, c

def add_numbers(a, b):
    return reduce_number((a, b))

def reduce_number(n):
    reduced = True
    while reduced:
        n, reduced, *_ = explode_rec(n, 0)
        if not reduced:
            n, reduced = split_rec(n)
    return n

def explode_rec(n, level):
    if not isinstance(n, int):
        l, r = n
        if level >= 4:
            return 0, True, l, r
        else:
            l, reduced, expl, expr = explode_rec(l, level + 1)
            if reduced:
                if expr != 0:
                    r = add_left(r, expr)
                    expr = 0
            else:
                r, reduced, expl, expr = explode_rec(r, level + 1)
                if reduced:
                    if expl != 0:
                        l = add_right(l, expl)
                        expl = 0
            if reduced:
                return (l, r), True, expl, expr
    return n, False, 0, 0

def add_left(n, m):
    if isinstance(n, int):
        return n + m
    else:
        a, b = n
        return add_left(a, m), b

def add_right(n, m):
    if isinstance(n, int):
        return n + m
    else:
        a, b = n
        return a, add_right(b, m)

def split_rec(n):
    if isinstance(n, int):
        if n >= 10:
            a = n // 2
            return (a, n - a), True
    else:
        l, r = n
        l, reduced = split_rec(l)
        if not reduced:
            r, reduced = split_rec(r)
        if reduced:
            return (l, r), True
    return n, False

def number_magnitude(n):
    if isinstance(n, int):
        return n
    l, r = n
    return 3 * number_magnitude(l) + 2 * number_magnitude(r)

In [ ]:
def part1(in_file):
    with open(in_file, 'r') as f:
        numbers = parse_numbers(map(str.strip, f))
        numbers = map(reduce_number, numbers)
        res = reduce(add_numbers, numbers)
    print(res)
    m = number_magnitude(res)
    print(m)

In [ ]:
part1('J18.csv')

In [ ]:
def part2(in_file):
    with open(in_file, 'r') as f:
        numbers = parse_numbers(map(str.strip, f))
        numbers = list(map(reduce_number, numbers))
    m_max = -np.Inf
    for i, n1 in enumerate(numbers):
        for j, n2 in enumerate(numbers):
            if i == j: continue
            m = number_magnitude(add_numbers(n1, n2))
            if m > m_max:
                m_max = m
    print(m_max)

In [ ]:
part2('J18.csv')

# J19

In [ ]:
df = pd.read_csv('J19.csv', sep=',', index_col=None, header=None, names=['x_scan', 'y_scan', 'z_scan'], dtype=str)
index_begin_scanner = list(df[df.y_scan.isna()].index)

In [ ]:
from bisect import bisect
df['scanner'] = df.index.map(lambda x : bisect(index_begin_scanner, x)-1)
df = df[df.y_scan.notna()].reset_index(drop=True)
df = df.astype('int32', copy=False)

In [ ]:
n_scanners = df.scanner.max()+1
n_scanned_beacons = df.shape[0]

dict_scanner = {}
for i in range(n_scanners):
    dict_scanner[i] = df.loc[df.scanner == i, ['x_scan', 'y_scan', 'z_scan']].to_numpy() 

In [ ]:
from collections import defaultdict
from itertools import combinations, product

# Matrices de rotations
rotations = [([2, 0, 1], [-1, -1, 1]), ([0, 1, 2], [1, -1, -1]), ([2, 1, 0], [-1, -1, -1]), ([2, 1, 0], [1, -1, 1]),
             ([0, 2, 1], [-1, -1, -1]), ([1, 2, 0], [1, -1, -1]), ([1, 0, 2], [-1, -1, -1]), ([1, 2, 0], [1, 1, 1]),
             ([0, 2, 1], [-1, 1, 1]), ([0, 1, 2], [-1, 1, -1]), ([0, 2, 1], [1, -1, 1]), ([2, 0, 1], [-1, 1, -1]),
             ([1, 0, 2], [1, 1, -1]), ([2, 1, 0], [1, 1, -1]), ([2, 0, 1], [1, 1, 1]), ([2, 1, 0], [-1, 1, 1]),
             ([0, 1, 2], [1, 1, 1]), ([1, 0, 2], [1, -1, 1]), ([1, 0, 2], [-1, 1, 1]), ([0, 1, 2], [-1, -1, 1]),
             ([1, 2, 0], [-1, 1, -1]), ([1, 2, 0], [-1, -1, 1]), ([0, 2, 1], [1, 1, -1]), ([2, 0, 1], [1, -1, -1])]


def part_1(scanners):
    intersects = get_intersections(scanners)
    mapping_dict = generate_mappings(intersects, scanners)
    beacons = set(tuple(p) for p in scanners[0])
    used_mappings = set()
    transformed_scanners = {0}
    scanner_origins = [[0, 0, 0]]
    while len(transformed_scanners) < len(scanners):
        queue = [k for k in mapping_dict.keys()
                 if k[0] in transformed_scanners and k[1] not in transformed_scanners]
        while len(queue) > 0:
            el = queue.pop()
            if el[1] in transformed_scanners:
                continue
            p_transpose = list(zip(*scanners[el[1]]))
            centroid = list(zip([0, 0, 0]))  # Origine relative du scanner 0, 0, 0
            use_mapping = el
            while True:
                centroid = transform(centroid, *mapping_dict[use_mapping])
                p_transpose = transform(p_transpose, *mapping_dict[use_mapping])
                new_points = set(tuple(p) for p in zip(*p_transpose))
                if use_mapping[0] == 0:
                    break
                for mapping in used_mappings:
                    if mapping[1] == use_mapping[0]:
                        use_mapping = mapping
                        break
            scanner_origins.append([centroid[0][0], centroid[1][0], centroid[2][0]])
            transformed_scanners.add(el[1])
            beacons.update(new_points)
            used_mappings.add(el)
    return len(beacons), scanner_origins


def part_2(scanner_origins: List):
    return max(sum(map(lambda x: abs(x[0] - x[1]), zip(*p))) for p in combinations(scanner_origins, 2))


def generate_mappings(intersects, scanners):
    mapping_dict = {}
    for i in intersects:
        p2dist_a = defaultdict(set)
        for p in combinations(scanners[i[0]], 2):
            dist = euclid_distance(*p)
            p2dist_a[tuple(p[0])].add(dist)
            p2dist_a[tuple(p[1])].add(dist)
        p2dist_b = defaultdict(set)
        for p in combinations(scanners[i[1]], 2):
            dist = euclid_distance(*p)
            p2dist_b[tuple(p[0])].add(dist)
            p2dist_b[tuple(p[1])].add(dist)
        points_a = []
        points_b = []
        for p in product(p2dist_a.keys(), p2dist_b.keys()):
            intersect = p2dist_a[p[0]].intersection(p2dist_b[p[1]])
            if len(intersect) >= 11:  # 12 common beacons 1 src and 11 dst for distance
                points_a.append([p[0][0], p[0][1], p[0][2]])
                points_b.append([p[1][0], p[1][1], p[1][2]])
        mapping_dict[i] = map_scanner_a_to_b(points_a, points_b)
    return mapping_dict


def map_scanner_a_to_b(points_a, points_b):
    a_transpose = list(zip(*points_a))
    b_transpose = list(zip(*points_b))
    for perms, signs in rotations:
        rotated = rotate(b_transpose, perms, signs)
        offset = []
        for p in zip(rotated, a_transpose):
            points = set([x[1] - x[0] for x in zip(p[0], p[1])])
            if len(points) == 1:
                offset.append(points.pop())
            if len(offset) == 3:
                return offset, perms, signs
    return None


def transform(to_transform, target_center, trans_perm, trans_sign):
    rotated = rotate(to_transform, trans_perm, trans_sign)
    return [list(map(lambda x: target_center[i] + x, p)) for i, p in enumerate(rotated)]


def get_intersections(scanners):
    intersections = []
    distance_dict = {i: set(euclid_distance(*p) for p in combinations(scanners[i], 2)) for i in scanners.keys()}
    for i in combinations(range(len(scanners)), 2):
        if len(distance_dict[i[0]].intersection(distance_dict[i[1]])) >= 66:  # 12!/10! / 2. Un peu de combinatoire
            intersections.append(i)
            intersections.append((i[1], i[0]))
    return intersections


def rotate(point, perms, signs):
    return map(lambda n: n * signs[0], point[perms[0]]), \
           map(lambda n: n * signs[1], point[perms[1]]), \
           map(lambda n: n * signs[2], point[perms[2]])


def euclid_distance(a, b):
    return np.sqrt(sum(map(lambda x: (x[0] - x[1])**2, zip(a, b))))

In [ ]:
part1, scanner_origin = part_1(dict_scanner)
part1

In [ ]:
part_2(scanner_origin)

# J20

In [ ]:
df = pd.read_csv('J20.txt', sep=' ', index_col=None, header=None, names=['x'])
algo = [{'.':0, '#':1}[c] for c in df.loc[0,'x']]
df = df.loc[1:]
df['x'] = df['x'].apply(lambda x:list(x))
n_cols = len(df.loc[1,'x'])
df = pd.DataFrame(df['x'].to_list(), columns=['x'+str(i) for i in range(n_cols)])
for c in df.columns:
    df[c] = df[c].map({'.':0, '#':1})
df = df.astype('int32', copy=False)
matrice = df.to_numpy()

In [ ]:
def enhance_img(n_runs, mat):
    mat_depart = np.pad(mat, ((n_runs+1,n_runs+1),(n_runs+1,n_runs+1)), mode='constant', constant_values=0)

    transform_from0 = 1
    transform_from1 = 0

    mat_courant = mat_depart.copy()

    for run in range(n_runs):
        if mat_courant[0,0]==0:
            mat_evol = np.full(mat_courant.shape, transform_from0, dtype=int)
        else:
            mat_evol = np.full(mat_courant.shape, transform_from1, dtype=int)

        for i in range(1, mat_courant.shape[0]-1): # on exclut les extrèmes
            for j in range(1, mat_courant.shape[1]-1):
                code = 0
                pow2 = 2**8
                for idelta in [-1,0,1]:
                    for jdelta in [-1,0,1]:
                        code += mat_courant[i+idelta, j+jdelta] * pow2
                        pow2 = int(pow2/2)
                mat_evol[i,j] = algo[code]
        mat_courant = mat_evol.copy()
    n = np.sum(mat_courant.sum())
    return n

Part 1

In [ ]:
enhance_img(2, matrice)

Part 2

In [ ]:
enhance_img(50, matrice)

# J21

In [ ]:
start_pos_players = [4,9]

Part 1

In [ ]:
current_score_players = [0,0]
current_pos_players = start_pos_players
rolling_dice = 0
player = 0

while True:
    somme = 3 * rolling_dice + (1+2+3)
    rolling_dice += 3
    current_pos_players[player] += + somme
    while (current_pos_players[player]>10):
        current_pos_players[player] -= 10
    current_score_players[player] += current_pos_players[player]
    if (current_score_players[player]>= 1000):
        break
    player = 1 - player

In [ ]:
current_score_players[1-player] * rolling_dice

Part 2

In [ ]:
sum_of3dices = {}
for i in range(1,4):
    for j in range(1,4):
        for k in range(1,4):
            if (i+j+k) in sum_of3dices:
                sum_of3dices[i+j+k] += 1
            else:
                sum_of3dices[i+j+k] = 1

In [ ]:
def dirac_dice(cur_pos1, cur_pos2, cur_score1, cur_score2):
    
    # retourne le vainqueur
    if cur_score1 >= 21: return (1, 0)
    if cur_score2 >= 21: return (0, 1)

    # Cumul de gains par joueur
    p1_wins_total = 0
    p2_wins_total = 0

    for tirage, nb_univers in sum_of3dices.items():
        
        cur_pos1_new = (cur_pos1 + tirage) 
        if cur_pos1_new > 10: cur_pos1_new -= 10 #ici on ne peut pas dépasser 19 à la base
        cur_score1_new = cur_score1 + cur_pos1_new

        # Tour de l'autre joueur
        p2_wins, p1_wins = dirac_dice(cur_pos2, cur_pos1_new, cur_score2, cur_score1_new)

        p1_wins_total += p1_wins * nb_univers
        p2_wins_total += p2_wins * nb_univers
    
    return (p1_wins_total, p2_wins_total)

max_victories = max(dirac_dice(start_pos_players[0], start_pos_players[1],0,0))


In [ ]:
max_victories

# J22

In [ ]:
df = pd.read_csv('J22.csv', sep=' ', index_col=None, header=None, names=['action', 'position'])

In [ ]:
df[['posx', 'posy', 'posz']] = df.position.str.split(',', expand=True)
df.posx = df.posx.str[2:]
df.posy = df.posy.str[2:]
df.posz = df.posz.str[2:]
df[['minx', 'maxx']] = df.posx.str.split('\.\.', expand=True).astype('int32', copy=False)
df[['miny', 'maxy']] = df.posy.str.split('\.\.', expand=True).astype('int32', copy=False)
df[['minz', 'maxz']] = df.posz.str.split('\.\.', expand=True).astype('int32', copy=False)
df = df[['action', 'minx', 'maxx', 'miny', 'maxy', 'minz', 'maxz']]

Part 1

In [ ]:
df2 = df.copy()

df2.minx = df2.minx.map(lambda x : min(51, max(-51, x)))
df2.maxx = df2.maxx.map(lambda x : min(51, max(-51, x)))
df2.miny = df2.miny.map(lambda x : min(51, max(-51, x)))
df2.maxy = df2.maxy.map(lambda x : min(51, max(-51, x)))
df2.minz = df2.minz.map(lambda x : min(51, max(-51, x)))
df2.maxz = df2.maxz.map(lambda x : min(51, max(-51, x)))


In [ ]:
cube = np.zeros((101,101,101))

In [ ]:
for i, row in df2.iterrows():
    for x in range(row.minx, row.maxx+1):
        for y in range(row.miny, row.maxy+1):
            for z in range(row.minz, row.maxz+1):
                if (-50 <= x <= 50) & (-50 <= y <= 50) & (-50 <= z <= 50):
                    cube[x+50, y+50, z+50] = 1 if row.action =='on' else 0

In [ ]:
cube.sum().sum().sum()

Part 2

In [ ]:
class Cube:
    def __init__(self, minx, maxx, miny, maxy, minz, maxz, statut):
        self.minx = minx
        self.maxx = maxx
        self.miny = miny
        self.maxy = maxy
        self.minz = minz
        self.maxz = maxz
        self.statut = statut
        self.size = (maxx - minx + 1) * (maxy - miny + 1) * (maxz - minz + 1)
    
    def __str__(self):
        return 'Cube : [' + str(self.minx) + ',' +  str(self.maxx) + '] - [' + str(self.miny) + ',' + str(self.maxy)  + '] - ['  + str(self.minz) + ',' + str(self.maxz) + '], size :' + str(self.size) + ', statut :' + str(self.statut)

    def __repr__(self):
        return str(self) + '\n'

In [ ]:
list_cubes = []
for i, row in df.iterrows():
    list_cubes.append(Cube(row.minx, row.maxx, row.miny, row.maxy, row.minz, row.maxz, 1 if row.action == 'on' else 0))

In [ ]:
def get_intersect(cube1, cube2, statut):
    minx = max(cube1.minx, cube2.minx)
    miny = max(cube1.miny, cube2.miny)
    minz = max(cube1.minz, cube2.minz)
    
    maxx = min(cube1.maxx, cube2.maxx)
    maxy = min(cube1.maxy, cube2.maxy)
    maxz = min(cube1.maxz, cube2.maxz)
    
    if (minx <= maxx) & (miny <= maxy) & (minz <= maxz):
        return Cube(minx, maxx, miny, maxy, minz, maxz, statut)
    
    return None

In [ ]:
def get_CubesToIndex(list_cubes, curCube, curIndex):
    if (curIndex >= len(list_cubes)):
        return 0
    if (curCube is None):
        return 0
    if (curCube.size == 0):
        return 0
    
    count = curCube.size * curCube.statut
    for i in range(curIndex):
        cube2 = list_cubes[i]
        overlap = get_intersect(curCube, cube2, cube2.statut)
        count -= get_CubesToIndex(list_cubes, overlap, i)

    return count

In [ ]:
count = 0
for i, cube in enumerate(list_cubes):
    count += get_CubesToIndex(list_cubes, cube, i);
count

# J23

In [ ]:
#Test

#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########

#state = 'BACDBCDA...........'

In [ ]:
#input

#############
#...........#
###D#D#B#A###
  #B#C#A#C#
  #########

state = 'DBDCBAAC...........'

In [ ]:
global_cost = {'A':1, 'B':10, 'C':100, 'D':1000}

In [ ]:
def swap(text, i, j):
    if i > j: i, j = j, i
    return text[:i] + text[j] + text[i + 1:j] + text[i] + text[j + 1:]   

In [ ]:
def walk(src, dst):
    step = -1 if src > dst else 1
    return range(src + step, dst + step, step)

In [ ]:
def into_hallway(state, door, rsize):
    first_door = 4 * rsize + 2
    doors = tuple(range(first_door, first_door + 8, 2))

    for end in (4 * rsize, len(state) - 1):
        for dist, i in enumerate(walk(door, end)):
            if state[i] != '.':
                break
            if i not in doors:
                yield dist + 1, i

In [ ]:
def steps(state, rsize):
    for room, expect in zip(range(0, 4 * rsize, rsize), 'ABCD'):
        door = 4 * rsize + 2 * (room // rsize + 1)
        for depth, cell in enumerate(state[room:room + rsize]):
            if cell != '.':
                if cell != expect or any(state[room + d] != expect
                                         for d in range(depth + 1, rsize)):
                    for door_dist, hall in into_hallway(state, door, rsize):
                        yield door_dist + depth + 1, room + depth, hall
                break

    for hall in range(4 * rsize, len(state)):
        apod = state[hall]
        if apod != '.':
            room = 'ABCD'.index(apod) * rsize
            door = 4 * rsize + 2 * (room // rsize + 1)
            if all(state[i] == '.' for i in walk(hall, door)):
                door_dist = abs(hall - door)
                for depth in range(rsize - 1, -1, -1):
                    cell = state[room + depth]
                    if cell == '.':
                        yield door_dist + depth + 1, hall, room + depth
                    elif cell != apod:
                        break

In [ ]:
def organize(state, rsize):
    work = [(0, state)]
    seen = {state: 0}
    target = ''.join(x * rsize for x in 'ABCD') + '...........'
    while work:
        energy, state = heapq.heappop(work)
        if state == target:
            return energy
        for dist, src, dst in steps(state, rsize):
            moved = swap(state, src, dst)
            moved_energy = energy + dist * global_cost[state[src]]
            if seen.get(moved, 1 << 32) > moved_energy:
                heapq.heappush(work, (moved_energy, moved))
                seen[moved] = moved_energy

In [ ]:
organize(state,2)

Part 2

In [ ]:
def extend(state, insert):
    return ''.join(state[2 * i] + ext + state[2 * i + 1]
                   for i, ext in enumerate(insert)) + state[8:]

In [ ]:
organize(extend(state, ('DD', 'CB', 'BA', 'AC')), 4)

# J24

In [ ]:
df = pd.read_csv('J24.csv', sep=' ', index_col=None, header=None, names=['instruction', 'a', 'b'])

Premier test

In [ ]:
def compute_instruction(df_loc, values_to_input, var_vals=None):
    if var_vals is None:
        var_vals = {'w':0, 'x':0, 'y':0, 'z': 0}
    n_imputed = 0
    for i, row in df_loc.iterrows():
        if row.instruction == 'inp':
            var_vals[row.a] = values_to_input[n_imputed]
            n_imputed += 1
        else:
            if row.b in var_vals:
                b = var_vals[row.b]
            else:
                b = int(row.b)
            if row.instruction == 'add':
                var_vals[row.a] += b
            if row.instruction == 'mul':
                var_vals[row.a] *= b
            if row.instruction == 'div':
                if b == 0:
                    return None
                var_vals[row.a] /= b
            if row.instruction == 'mod':
                if (var_vals[row.a] < 0) | (b <= 0):
                    return None
                var_vals[row.a] = var_vals[row.a] - b * (var_vals[row.a] // b)
            if row.instruction == 'eql':
                var_vals[row.a] = 1 if var_vals[row.a] == b else 0
    
    return var_vals

In [ ]:
value_to_test_lst = [9,9,9,9,9,9,9,9,9,9,9,9,9,9]
value_to_test = 99999999999999

Deuxième test

In [ ]:
input_index = list(df[df.instruction == 'inp'].index)
input_index.append(df.shape[0])

In [ ]:
len(input_index)

In [ ]:
dict_df = {}
for i in range(len(input_index)-1):
    ind = input_index[i]
    ind_next = input_index[i+1]
    dict_df[i] = df.loc[ind+1:ind_next-1]

In [ ]:
def sign(a):
    return 1 if a > 0 else -1 if a < 0 else 0

def div(x, y):
    return abs(x) // abs(y) * sign(x) * sign(y)

def mod(x, y):
    return x - div(x, y) * y

In [ ]:
def simplify(df_instructions, w, x, y, z, trace=False):
    regs = {"w": w, "x": x, "y": y, "z": z}
    for i, row in df_instructions.iterrows():
        op = row.instruction
        left = row.a     
        right = row.b if row.b in 'wxyz' else int(row.b)
        a = regs[left]
        b = right if isinstance(right, int) else regs[right]
        if op == "add":
            if isinstance(a, int) and isinstance(b, int):
                regs[left] = a + b
            elif b == 0:
                regs[left] = a
            elif a == 0:
                regs[left] = b
            else:
                regs[left] = ("add", a, b)
        elif op == "mul":
            if isinstance(a, int) and isinstance(b, int):
                regs[left] = a * b
            elif a == 0 or b == 0:
                regs[left] = 0
            elif a == 1:
                regs[left] = b
            elif b == 1:
                regs[left] = a
            else:
                regs[left] = ("mul", a, b)
        elif op == "div":
            if isinstance(a, int) and isinstance(b, int):
                regs[left] = div(a, b)
            elif a == 0:
                regs[left] = 0
            elif b == 1:
                regs[left] = a
            else:
                regs[left] = ("div", a, b)
        elif op == "mod":
            if isinstance(a, int) and isinstance(b, int):
                regs[left] = mod(a, b)
            elif a == 0:
                regs[left] = 0
            elif b == 1:
                regs[left] = 0
            else:
                regs[left] = ("mod", a, b)
        elif op == "eql":
            if isinstance(a, int) and isinstance(b, int):
                regs[left] = int(a == b)
            elif (
                isinstance(b, int)
                and (b < 1 or b > 9)
                and isinstance(left, str)
                and left.startswith("w")
            ):
                regs[left] = 0
            elif (
                isinstance(a, int)
                and (a < 1 or a > 9)
                and isinstance(right, str)
                and right.startswith("w")
            ):
                regs[left] = 0
            elif b == 0 and isinstance(a, tuple) and a[0] == "eql":
                regs[left] = ("neq", a[1], a[2])
            else:
                regs[left] = ("eql", a, b)
        else:
            raise RuntimeError(f"Invalid {op} {left} {right} {a} {b}")
        if trace:
            print((op, left, right, regs))
    return regs["x"], regs["y"], regs["z"]


def to_string(z):
    if isinstance(z, (int, str)):
        return str(z)
    op, a, b = z
    if op == "neq":
        return f"({to_string(a)}!={to_string(b)})"
    if op == "eql":
        return f"({to_string(a)}=={to_string(b)})"
    if op == "add":
        if isinstance(b, int) and b < 0:
            return f"({to_string(a)}-{-b})"
        return f"({to_string(a)}+{to_string(b)})"
    if op == "mul":
        return f"{to_string(a)}*{to_string(b)}"
    if op == "div":
        return f"({to_string(a)}//{to_string(b)})"
    if op == "mod":
        return f"({to_string(a)}%{to_string(b)})"


def z_target(df_instructions):
    for i, row in df_instructions.iterrows():
        op = row.instruction
        a = row.a     
        b = row.b if row.b in 'wxyz' else int(row.b)
        if op == "add" and a == "x" and isinstance(b, int):
            return -b if b < 0 else None
    return None

In [ ]:
states = {}
result = {}
for w in range(1, 10):
    x, y, z = simplify(dict_df[0], w, 0, 0, 0)
    result[z] = (w,)
print(f"Step 0 z={min(result)}..{max(result)} {len(result)}")

for index, b in dict_df.items():
    if index == 0: continue
    prev = result
    result = {}
    dec = 0
    z_targ = z_target(b)
    for w in range(9,0,-1): #fort part 1
    #for w in range(1, 10): #for part 2
        for z in prev:
            if z_targ is not None and z % 26 - z_targ != w:
                continue
            x, y, z1 = simplify(b, w, 0, 0, z)
            if z1 not in result:
                result[z1] = prev[z] + (w,)
                if z1 < z:
                    dec += 1
    print(to_string(simplify(b, "w", "x", "y", "z")[2]))
    print(f"Step {index} z={min(result)}..{max(result)} {len(result)} {dec=}")

"".join(str(n) for n in result[0])

# J25

In [11]:
with open("J25.txt", "r", encoding='UTF-8') as file:
    input_list_in = [str(line.strip()) for line in file]
    w, h = (len(input_list_in[0]), len(input_list_in))

Part 1

In [12]:
m = 0
finished = False
while not finished:
    finished = True # assume finished - no fish moved yet

    # East swimmers
    input_list_new = [] # List of strings after East swims
    for s in input_list_in:
        if s[w-1] + s[0] == '>.':
            s = 'p' + s[1:w-1] + ':' # fill in after other '>.' -> '.>' moves
        input_list_new += [s.replace('>.','.>').replace(':','.').replace('p','>')]
        if input_list_new[-1] != s: finished = False

    # South swimmers
    input_list_in = [''.join(i) for i in zip(*input_list_new)] # transpose

    input_list_new = [] # List of strings after South swims (transposed)
    for s in input_list_in:
        if s[h-1] + s[0] == 'v.':
            s = 'p' + s[1:h-1] + ':' # fill in after other 'v.' -> '.v' moves
        input_list_new += [s.replace('v.','.v').replace(':','.').replace('p','v')]
        if input_list_new[-1] != s: finished = False

    input_list_in = [''.join(i) for i in zip(*input_list_new)] # transpose again to restore

    m += 1
    
print('Number of moves:', m)

Number of moves: 432
